In [1]:
# !pip install upgrade google-api-python-client

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

Once we have our credentials set up, we can now start collecting comments! We’ll first build the service for calling the YouTube API:

In [ ]:
from apiclient.discovery import build

def build_service(filename):
    with open(filename) as f:
        key = f.readline()

    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [ ]:
service = build_service('apikey.json') 

In [ ]:
df = pd.read_csv('archive/US_youtube_trending_data.csv')
df = df.head(50)
videoIdList = df['video_id'].tolist()
trendingDateList = df['trending_date'].tolist()
# videoIdList
# trendingDateList

In [ ]:
#test code
video_trending_date = trendingDateList[1]
date_format = "%Y-%m-%dT%H:%M:%SZ"
a = datetime.strptime(video_trending_date, date_format)
b = datetime.strptime(item['snippet']['topLevelComment']['snippet']['publishedAt'], date_format)
print((a-b).days)

-251


In [ ]:
# videoIdList = ['9bZkp7q19f0']
# videoIdList = ['3C66w5Z0ixs','M9Pmf9AB4Mo','J78aPJ3VyNs','kXLn3HkpjaA','VIUo6yapDbc','w-aidBdvZo8','uet14uf9NsE',
#                'ua4QMFQATco','SnsPZj91R7E','SsWHMAhshPQ','49Z6Mv4_WCA','nt3VVyv5pxQ','I6hswz4rIrU','W7VK4DUHvKU',
#                'W9Aen8hG20Y','BNeDH6UTmXw','6TIsR_7nrNc','gPdUslndvVI','GTp-0S82guE']
comment_count = 2000

parse for the whole list of youtube videos
	for each video
		make a service call for 100 responses of top comments
		store the data in list
		
		make the service call 9 time for 900 more responses
		store in list
		
		create dataframe of lists
		write in csv, file name comment id

In [ ]:
def fetch_comments_teratively(videoId, comment_count):
    comment, comment_id, reply_count, like_count, published_date = [], [], [], [], []
    first_response = True
    video_trending_date = trendingDateList[videoIdList.index(videoId)]
    if (first_response):

        response = service.commentThreads().list(
            part='snippet',
            maxResults=100,
            textFormat='plainText',
            order='time',
            videoId=videoId
        ).execute()
        first_response = False
    
#     comment, comment_id, reply_count, like_count = add_data_to_lists(response, comment, comment_id, reply_count, like_count)
    for item in response['items']:
        date_format = "%Y-%m-%dT%H:%M:%SZ"
        a = datetime.strptime(video_trending_date, date_format)
        b = datetime.strptime(item['snippet']['topLevelComment']['snippet']['publishedAt'], date_format)
        delta = a - b
        if(delta.days <= 3 and delta.days >= 0):
            comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
            comment_id.append(item['snippet']['topLevelComment']['id'])
            reply_count.append(item['snippet']['totalReplyCount'])
            like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
            published_date.append(item['snippet']['topLevelComment']['snippet']['publishedAt'])
    
#     print(len(comment))
    while(len(comment) < comment_count):
        try:
            if(response['nextPageToken'] != None):
#                 print(response['nextPageToken'])
                response = service.commentThreads().list(
                    part='snippet',
                    maxResults=100,
                    textFormat='plainText',
                    order='time',
                    videoId=videoId,
                    pageToken=response['nextPageToken']
                ).execute()

                for item in response['items']:
                    date_format = "%Y-%m-%dT%H:%M:%SZ"
                    a = datetime.strptime(video_trending_date, date_format)
                    b = datetime.strptime(item['snippet']['topLevelComment']['snippet']['publishedAt'], date_format)
                    delta = a - b
                    if(delta.days <= 3 and delta.days >= 0):
                        comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
                        comment_id.append(item['snippet']['topLevelComment']['id'])
                        reply_count.append(item['snippet']['totalReplyCount'])
                        like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
                        published_date.append(item['snippet']['topLevelComment']['snippet']['publishedAt'])
        except:
            print("Cannot get NextPageToken for interation and Video Id", i,videoId)
            break
    print(videoId,len(comment))
    first_response = True
    return comment, comment_id, reply_count, like_count, published_date

In [2]:
for i in range(0, len(videoIdList)):
    try:
        comment, comment_id, reply_count, like_count, published_date = fetch_comments_teratively(videoIdList[i], comment_count)
        comments = pd.DataFrame(list(zip(comment, comment_id, reply_count, like_count, published_date)),
                       columns =['comment', 'comment_id', 'reply_count', 'like_count', 'published_date'])
        if not comments.empty:
            comments.to_csv(r"comment_data/"+videoIdList[i]+".csv", index=False)
    except:
        print("Failed on this video, trying next...", videoIdList[i])
        continue

In [ ]:
#test code
print(len(comment))

1881


In [3]:
#test code
response = service.commentThreads().list(
            part='snippet',
            maxResults=100,
            textFormat='plainText',
            order='time',
            videoId=videoIdList[1]
        ).execute()

response_list = []
for item in response['items']:
    response_list.append(item)
    print(item)
    
# rawdata_df = pd.DataFrame(item, columns=["Commentdata"])
# rawdata_df.to_csv(r"comment_data/rawdata_comments.csv", index=False)

In [ ]:
#test code
rawdata_df.head()

,Commentdata
